Import Libraries
TO DO
1) Describe Grid Location bottom left hand location and Cell Size
2) Plot the vector using x y coordinates.
3) Compare to Raster Stat Results, need to make sure that it only includes the cells fully inside (not on the border of the vector.

In [219]:
# TO DO
## 1) switch all cords to y,x in some places it is x,y and others y,x make it consistent with how the grid is stored (y,x)
## 2) Fix edge case where the top row (where y index > 0) causing numpy to wrap to the bottom of Y

In [220]:
import numpy as np
print (np.__version__)
from osgeo import gdal
from scipy import stats
import timeit
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
from math import ceil,floor

1.11.3


<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [221]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
shape = "data\\test.shp"
g = np.array(r.GetRasterBand(1).ReadAsArray())
csze = r.GetGeoTransform()[1]
ulX = r.GetGeoTransform()[0]
ulY = r.GetGeoTransform()[3]

Function to create data structure function takes in a grid with mxn and returns a grid of the same size summed by columns.  A vectorized version would be faster

In [224]:
## Function to create the output dataset
def dtFZS(g,out):
    for xw in range(0,g.shape[1]):
        for yw in range(0,g.shape[0]):
            out[yw,xw] = out[yw-1,xw] + g[yw,xw]

In [225]:
##Create The Output Summed Down DataSet
DS_FstZS = np.zeros(g.shape,dtype='uint64')
dtFZS(g,DS_FstZS)


Create Helper Functions to move between world and grid cordinates

In [226]:
## Input (world cords; x1, x2, Grid Upper Left, and Cellsize)
## Returns (grid cords; list(x))
## x1 and x2 represent vertexes on a polygon in order

def x_world_to_grid(wx1,wx2,ulX,clSze):
    x1a = min(wx1,wx2)
    x2a = max(wx1,wx2)
    
    xstart = ceil  ((x1a - ulX - (clSze/2)) / clSze)
    xstop =  floor ((x2a - ulX - (clSze/2)) / clSze)

    return(list(range(xstart,xstop+1)))

In [227]:
## Input (grid cords; list(x))
## Returns(world cords centroid ;(list(X)))

def x_grid_to_world(gx,ulX,clSze):
    return [i * clSze + ulX + (clSze/2) for i in gx]

In [228]:
## Input(world cords centroid ;(list(X)))
## Returns(world cords centroid ;(list(y)))

def y_world_pred(wx1,wx2,wy1,wy2,wxpred):
    slope = 0;
    if (wx1 > wx2): ## switch Coords
        wx1,wx2,wy1,wy2 = wx2,wx1,wy2,wy1
    if(wx1 != wx2):    
        slope = (wy2-wy1)/(wx2-wx1)
    ##return [ i for i in xpred]
    return [(slope * (i-wx1)) + wy1 for i in wxpred]

In [229]:
## Input(world cords centroid ;(list(y)))
## Returns (grid cords; list(y))

def y_world_to_grid(ulY,clSze,wypred):
        return [floor((ulY - i - (clSze/2)) / clSze ) for i in wypred] 

In [230]:
## Combine all the helper functions to return list of cells
## Input(world Cords for 2 vertexes on a polygon, TL corner of grid, and cellsize)
## Returns(y,x grid cords)
## counts if centroid is in poylgon

def xy_grid(wx1,wx2,ulX,wy1,wy2,ulY,clSze):
    print ("wx1 = " + str(wx1) + " wx2 = " + str(wx2) + " ulx = " +str(ulX))
    print ("wy1 = " + str(wy1) + " wy2 = " + str(wx2) + " uly = " +str(ulY))
    print("cellsize =" + str(clSze))
    xgrid  = x_world_to_grid(wx1,wx2,ulX,clSze)
    wxpred = x_grid_to_world(xgrid,ulX,clSze)  
    wypred = y_world_pred(wx1,wx2,wy1,wy2,wxpred)
    ygrid  = y_world_to_grid(ulY,clSze,wypred)
    ##return xgrid
    return((ygrid,xgrid))


In [231]:
## Input(list of cords in world, summed down grid, real-world grid tl, cllSze)
## Output Sum v in polygon

def FstZS(v,DS_FstZS,ulX,ulY,clSze):
    total = 0;    
    for p in range(1,(len(v)+0)):
        ## If moving left
        if v[p][0] > v[p-1][0]:
            cords = xy_grid(v[p][0],v[p-1][0],ulX,v[p][1],v[p-1][1],ulY,clSze)
            ##print('add')
            ##print(cords)
            ##print(np.sum(DS_FstZS[cords]))
            total += np.sum(DS_FstZS[cords])
        ## If moving Right
        elif v[p][0] < v[p-1][0]: 
            cords = xy_grid(v[p][0],v[p-1][0],ulX,v[p][1],v[p-1][1],ulY,clSze)
            ##print('sub')
            ##print (cords)
            ##print(np.sum(DS_FstZS[cords]))
            total -= np.sum(DS_FstZS[cords])
    return int(total)

In [242]:
## Some testing Data

tlx,tly,blx,bly = 1608590,2038038,1608450,2038138
##tlx,tly,blx,bly = 1608490,2038038,1608450,2038138
##tlx,tly,blx,bly = 1599639.1,2083280,1599683.4,2083210
##tlx,tly,blx,bly = 1599639.1,2083280,1599683.4,2083210

## Just the top left hand cell
## This is an edge case as it wraps around to the bottom TO DO 2)
##tlx,tly,blx,bly = 1599636,2083302,1599649,2083289

## Bottom edge case Pass
#tlx,tly,blx,bly = 1674639.9,2031308.0,1674659.7,2031290.9

## Bigger Test Case
tlx,tly,blx,bly = 1610584,2069215,1665119,2042733

v = [(tlx,tly),(tlx,bly),(blx,bly),(blx,tly),(tlx,tly)]

In [243]:
#g.shape
test = FstZS(v,DS_FstZS,ulX,ulY,csze)
test

wx1 = 1665119 wx2 = 1610584 ulx = 1599633.055684202
wy1 = 2042733 wy2 = 1610584 uly = 2083307.9820680085
cellsize =30.0
wx1 = 1610584 wx2 = 1665119 ulx = 1599633.055684202
wy1 = 2069215 wy2 = 1665119 uly = 2083307.9820680085
cellsize =30.0


82877158

In [244]:
### Import Test vector
poly = Polygon(v) 
zonal_stats(poly,"data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif",stats= "sum")

C:\Users\smh362\AppData\Local\Continuum\Anaconda3\lib\site-packages\rasterstats\main.py:145: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
C:\Users\smh362\AppData\Local\Continuum\Anaconda3\lib\site-packages\rasterstats\io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)


[{'sum': 82877158.0}]

In [238]:
## To do replace with a better test
def test(v):
    rs =  zonal_stats(Polygon(v),"data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif",stats= "sum")
    nv =  FstZS(v,DS_FstZS,ulX,ulY,csze)
    return rs[0]['sum'] - nv

In [157]:
##test(v)